# QueryPolicy Examples

# Import to run %sh magic cell

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

# Add Java Client POM Dependency

In [2]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>7.2.1</version>
  </dependency>
</dependencies>

# Add required Imports

In [3]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
System.out.println("Client modules imported.");

Client modules imported.


# Generate Test Data

In [4]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

In [5]:
%sh aql -c "set key_send true"

In [6]:
%sh aql -f "../aqlScripts/insert.aql"

In [7]:
AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");

Key key = new Key("test", "testset", "key1");
System.out.println("Working with record key:");
System.out.println(key);

Record record = client.get(null, key);
System.out.println("Read back the record.");

System.out.println("Record values are:");
System.out.println(record);

Initialized the client and connected to the cluster.
Working with record key:
test:testset:key1:bf6c1d13e7cd10c5bd022d27e7df170c0bccd6e1
Read back the record.
Record values are:
(gen:1),(exp:450911703),(bins:(name:Jack),(age:26))


In [8]:
%sh asadm --enable -e "manage sindex create numeric idx_age ns test set testset bin age"

In [9]:
//Needed imports
import com.aerospike.client.query.Statement;
import com.aerospike.client.query.Filter;
import com.aerospike.client.Operation;
import com.aerospike.client.Bin;
import com.aerospike.client.policy.QueryPolicy;
import com.aerospike.client.query.RecordSet;

//Run SI query
Statement stmt = new Statement();
stmt.setNamespace("test");
stmt.setSetName("testset");
stmt.setFilter(Filter.range("age", 20,60));
//Operation[] ops = new Operation[1];
//Bin ageBinNull = Bin.asNull("age");
//ops[0] = Operation.put(ageBinNull);
//stmt.setOperations(ops);
QueryPolicy qp = new QueryPolicy();
//qp.recordQueueSize = 5000;
RecordSet rs = client.query(qp, stmt);

WritePolicy wp = new WritePolicy();

int nCount = 0;

while(rs.next()){
    Record r = rs.getRecord();
    Key thisKey = rs.getKey();  
    System.out.println(r);
    System.out.println(thisKey);
    Bin newBin = new Bin("newBin", nCount);
    nCount++;
    client.put(wp, thisKey, newBin);
    System.out.println(client.get(null,thisKey));
}



(gen:1),(exp:450911703),(bins:(name:Sean),(age:24))
test:testset:key7:1ab5bb58a1d1e17dcf398a8039ddd09f7460606a
(gen:2),(exp:450911762),(bins:(name:Sean),(age:24),(newBin:0))
(gen:1),(exp:450911703),(bins:(name:Sally),(age:32))
test:testset:key6:f7088d6a1333cd392d26eb832e321730be365150
(gen:2),(exp:450911762),(bins:(name:Sally),(age:32),(newBin:1))
(gen:1),(exp:450911703),(bins:(name:Sandra),(age:34))
test:testset:key0:18eb429f44c1639dd4c50a02fbd380640ead86b4
(gen:2),(exp:450911762),(bins:(name:Sandra),(age:34),(newBin:2))
(gen:1),(exp:450911703),(bins:(name:Jill),(age:20))
test:testset:key2:c10b2c764e604c439039804ec947fa2b18520d1a
(gen:2),(exp:450911762),(bins:(name:Jill),(age:20),(newBin:3))
(gen:1),(exp:450911703),(bins:(name:Jim),(age:46))
test:testset:key4:703ca4a03068c57171bc4be1f829fcdd604ccdc5
(gen:2),(exp:450911762),(bins:(name:Jim),(age:46),(newBin:4))
(gen:1),(exp:450911703),(bins:(name:Jack),(age:26))
test:testset:key1:bf6c1d13e7cd10c5bd022d27e7df170c0bccd6e1
(gen:2),(exp:45

# Cleanup

In [10]:
%sh asadm --enable -e "manage sindex delete idx_age ns test set testset"

In [11]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"